# Aluno: Vinícius Oliveira Fernandes
# Ponderada 04 - Análise de Testes A/B em Simulação de Preferências de Jogos

## Implementação 2 de Teste A/B em Simulador de Preferências de Jogos

In [6]:
import random
import math
from scipy import stats

class Usuario:
    def __init__(self, idade, genero, preferencias_genero, tempo_medio_jogo):
        self.idade = idade
        self.genero = genero
        self.preferencias_genero = preferencias_genero
        self.tempo_medio_jogo = tempo_medio_jogo

class Jogo:
    def __init__(self, nome, genero, faixa_etaria, tempo_medio_jogo):
        self.nome = nome
        self.genero = genero
        self.faixa_etaria = faixa_etaria
        self.tempo_medio_jogo = tempo_medio_jogo

# Função para gerar dados simulados de usuários
def gerar_dados_usuarios(n):
    idades = [random.randint(18, 35) for _ in range(n)]
    generos = ["Masculino" if random.random() < 0.5 else "Feminino" for _ in range(n)]
    preferencias_genero = [["Ação"] if random.random() < 0.5 else ["Aventura"] for _ in range(n)]
    tempo_medio_jogo = [random.randint(30, 90) for _ in range(n)]

    return [Usuario(idades[i], generos[i], preferencias_genero[i], tempo_medio_jogo[i]) for i in range(n)]

# Gerando uma quantidade maior de dados de usuários
usuarios = gerar_dados_usuarios(100)
jogos = [Jogo("Jogo A", "Ação", 18, 40), Jogo("Jogo B", "Aventura", 12, 30)]

def escolher_jogo(usuario, jogos):
    jogos_preferidos = [jogo for jogo in jogos if jogo.genero in usuario.preferencias_genero]
    return random.choice(jogos_preferidos) if jogos_preferidos else None

# Função para calcular o tamanho da amostra necessário
def calcular_tamanho_amostra(variancia, margem_erro, z_score):
    return (z_score ** 2 * variancia) / (margem_erro ** 2)

# Implementação do Teste A/B
def aplicar_teste_ab():
    # Alterar o tempo médio de jogo do "Jogo A"
    for jogo in jogos:
        if jogo.nome == "Jogo A":
            jogo.tempo_medio_jogo = 60  # Nova duração do jogo para o teste A/B

# Coleta de dados para a fase antes da alteração
def coletar_dados_teste_ab():
    escolhas_antes = [escolher_jogo(usuario, jogos) for usuario in usuarios]

    # Aplicar a alteração
    aplicar_teste_ab()

    escolhas_depois = [escolher_jogo(usuario, jogos) for usuario in usuarios]

    return escolhas_antes, escolhas_depois

# Análise dos resultados
def analisar_resultados(escolhas_antes, escolhas_depois):
    escolhas_antes_contagem = {jogo.nome: 0 for jogo in jogos}
    escolhas_depois_contagem = {jogo.nome: 0 for jogo in jogos}

    for escolha in escolhas_antes:
        if escolha:
            escolhas_antes_contagem[escolha.nome] += 1

    for escolha in escolhas_depois:
        if escolha:
            escolhas_depois_contagem[escolha.nome] += 1

    return escolhas_antes_contagem, escolhas_depois_contagem

# Função para calcular o p-valor
def calcular_p_valor(contagem_antes, contagem_depois):
    p_valores = {}
    for jogo in contagem_antes.keys():
        sucesso_antes = contagem_antes[jogo]
        sucesso_depois = contagem_depois[jogo]
        total_antes = sum(contagem_antes.values())
        total_depois = sum(contagem_depois.values())

        if total_antes == 0 or total_depois == 0:
            p_valores[jogo] = 1.0
            continue

        # Calculando as proporções
        proporcao_antes = sucesso_antes / total_antes
        proporcao_depois = sucesso_depois / total_depois

        # Calculando o p-valor usando o teste z
        p_valores[jogo] = stats.norm.sf(abs(proporcao_antes - proporcao_depois) / math.sqrt((proporcao_antes * (1 - proporcao_antes) / total_antes) + (proporcao_depois * (1 - proporcao_depois) / total_depois)))

    return p_valores

# Relatório dos resultados do Teste A/B
def relatorio_resultados(escolhas_antes_contagem, escolhas_depois_contagem, p_valores):
    print("Relatório de Resultados do Teste A/B")
    print("===================================")
    print("Escolhas Antes da Alteração:")
    for jogo, contagem in escolhas_antes_contagem.items():
        print(f"{jogo}: {contagem} escolhas")

    print("\nEscolhas Depois da Alteração:")
    for jogo, contagem in escolhas_depois_contagem.items():
        print(f"{jogo}: {contagem} escolhas")

    print("\nAnálise:")
    for jogo in escolhas_antes_contagem.keys():
        diferenca = escolhas_depois_contagem[jogo] - escolhas_antes_contagem[jogo]
        print(f"O jogo {jogo} teve uma variação de {diferenca} escolhas após a alteração.")
        print(f"P-valor para {jogo}: {p_valores[jogo]}")

# Função principal que executa o teste A/B
def main():
    escolhas_antes, escolhas_depois = coletar_dados_teste_ab()
    escolhas_antes_contagem, escolhas_depois_contagem = analisar_resultados(escolhas_antes, escolhas_depois)
    p_valores = calcular_p_valor(escolhas_antes_contagem, escolhas_depois_contagem)
    relatorio_resultados(escolhas_antes_contagem, escolhas_depois_contagem, p_valores)

if __name__ == "__main__":
    main()

Relatório de Resultados do Teste A/B
Escolhas Antes da Alteração:
Jogo A: 48 escolhas
Jogo B: 52 escolhas

Escolhas Depois da Alteração:
Jogo A: 48 escolhas
Jogo B: 52 escolhas

Análise:
O jogo Jogo A teve uma variação de 0 escolhas após a alteração.
P-valor para Jogo A: 0.5
O jogo Jogo B teve uma variação de 0 escolhas após a alteração.
P-valor para Jogo B: 0.5
